# Preselection 

## Import libraries and choose whether to run locally or distributed

In [1]:
import ROOT
import os
from dask.distributed import Client
distributed = False
MT = False

Error in <TExMap::Add>: key 140014193245664 is not unique
Error in <TExMap::Add>: key 140014193762256 is not unique
TClassTable::Add:0: RuntimeWarning: class timespec already in TClassTable
TClassTable::Add:0: RuntimeWarning: class reverse_iterator<__gnu_cxx::__normal_iterator<TString*,vector<TString> > > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class reverse_iterator<__gnu_cxx::__normal_iterator<int*,vector<int> > > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class reverse_iterator<__gnu_cxx::__normal_iterator<string*,vector<string> > > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<char*,string> already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<const char*,string> already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<TString*,vector<TString> > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__no

Welcome to JupyROOT 6.25/01


In [2]:
if distributed != True and MT == True:
    ROOT.ROOT.EnableImplicitMT()

## Declare utils defined in a preselection.h file to the ROOT interpreter. If running distributed, connect to cluster and declare the file content in an initialization function

In [3]:
def my_initialization_function():
    ROOT.gInterpreter.Declare('{}'.format(data))

In [4]:
if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    sched_port = 38651
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    text_file = open("preselection.h", "r")
    data = text_file.read()
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)

else:
    RDataFrame = ROOT.RDataFrame
    ROOT.gInterpreter.Declare('#include "preselection.h"')

Loading histogram NUM_TightID_DEN_genTracks_eta_pt from file https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/leptonSF/Mu_RunBCDEFGH_SF_ID_2016_syst.root... 
Loading histogram EGamma_SF2D from file https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/leptonSF/EGM2D_RECO_SF_2016.root... 
Loading histogram EGamma_SF2D from file https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/leptonSF/2016LegacyReReco_ElectronMVA90noiso_Fall17V2.root... 
Loading histogram NUM_TightID_DEN_genTracks_pt_abseta from file https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/leptonSF/Muon_RunBCDEF_SF_ID_2017.root... 
Loading histogram NUM_TightRelIso_DEN_TightIDandIPCut_pt_abseta from file https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools/python/postprocessing/data/leptonSF/Muon_RunBCDEF_SF_ISO_2017.root... 
Loading histogram EGamma_SF2D from file https://ttedesch.web.cern.ch/ttedesch/nanoAOD-tools

## Define the dataframe starting from a chain of root files

In [5]:
!xrdcp root://xrootd-cms.infn.it//store/mc/RunIIFall17NanoAODv7/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/100000/5ADAED4E-FEEE-884F-8555-FF6FB31D6A60.root .

[0B/0B][100%][==================================================][0B/s]  
Run: [ERROR] Server responded with an error: [3018] file exists (destination)



In [6]:
#proxy = "root://212.189.205.221:30443//root://xrootd-cms.infn.it/"
proxy = "root://xrootd-cms.infn.it/"

chain = [proxy + "/store/mc/RunIIFall17NanoAODv7/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/100000/5ADAED4E-FEEE-884F-8555-FF6FB31D6A60.root"]
#chain = ["RunIIFall17NanoAODv7-5ADAED4E-FEEE-884F-8555-FF6FB31D6A60.root"]


if distributed == True:
    df = RDataFrame("Events", chain, npartitions=4, daskclient=client)
else:
    df = RDataFrame("Events", chain)

## Define preselection flow

In [7]:
df_year = df.DefinePerSample("Year","GetYear(rdfslot_, rdfsampleinfo_)")

#### MCWeight_writer #### #to check
'''
for s in samples:
    df_sample[s] = df_year.Filter("Sample = XX")
    nevents_df[s] = df_sample[s].Sum("1")
    gen_weight_df[s] = df_sample[s].Sum("Generator_weight")
    q2_weight[s][df_sample[s].Sum("LHEScaleWeight[{}]").format(j) for j in len(0,7)]
    ps_weight[s][df_sample[s].Sum("PSWeight[{}]").format(j) for j in len(0,4)]
    pdf_weight[s][df_sample[s].Sum("W_Pdf[{}]").format(j) for j in len(0,15)]
'''

#### MET HLT Filter ####   #### working only for 2017 and 2018 (2016 requires a column that is not present in 2017 nd 2018, i.e. HLT_IsoTkMu24
df_METHLTFilter = df_year.Define("METHLTFilter", "MET_HLT_Filter(Year, Flag_goodVertices, Flag_HBHENoiseFilter, Flag_HBHENoiseIsoFilter, Flag_EcalDeadCellTriggerPrimitiveFilter, Flag_BadPFMuonFilter, Flag_globalSuperTightHalo2016Filter, HLT_Ele27_WPTight_Gsf,  HLT_Ele32_WPTight_Gsf, HLT_IsoMu24, HLT_IsoMu27, HLT_Mu50, HLT_Ele35_WPTight_Gsf, HLT_Ele32_WPTight_Gsf_L1DoubleEG, HLT_Photon200)")\
                         .Filter("METHLTFilter == true", "MET HLT Filter")

#### preselection #####
df_preselection = df_METHLTFilter.Filter("PV_ndof> 4 && abs(PV_z) < 20 && hypot(PV_x, PV_y)<2", "Good vertex")\
                                 .Define("HT_eventHT", "GetEventHT(Jet_pt, Jet_eta, Jet_phi, Jet_mass)")

#### lepSF #####
df_LepSF = df_preselection.Define("LepSF_output", "LepSF(Electron_pt, Electron_eta, Electron_pdgId, Muon_pt, Muon_eta, Muon_pdgId, Year)")\
                          .Define("Muon_effSF", "LepSF_output[0]")\
                          .Define("Electron_effSF", "LepSF_output[1]")\
                          .Define("Muon_effSF_errUp", "LepSF_output[2]")\
                          .Define("Electron_effSF_errUp", "LepSF_output[3]")\
                          .Define("Muon_effSF_errDown", "LepSF_output[4]")\
                          .Define("Electron_effSF_errDown", "LepSF_output[5]")

#### mht ####
df_mht = df_LepSF.Define("MHT_pt_phi", "MHT_pt_phi(Electron_pt, Electron_eta, Electron_phi, Electron_mass, Electron_miniPFRelIso_all, Muon_pt, Muon_eta, Muon_phi, Muon_mass, Muon_miniPFRelIso_all, Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_muonIdx1, Jet_muonIdx2, Jet_electronIdx1, Jet_electronIdx2, nJet)")\
                 .Define("MHT_pt", "MHT_pt_phi[0]").Define("MHT_phi", "MHT_pt_phi[1]")\

#### puWeight #####
df_puWeight = df_mht.Define("puWeights", "puWeight(Year, Pileup_nTrueInt, IsMC)")\
                    .Define("puWeight", "puWeights[0]")\
                    .Define("puWeightUp", "puWeights[1]")\
                    .Define("puWeightDown", "puWeights[2]")

#### prefCorr ####
df_prefCorr = df_puWeight.Define("prefCorrs","PrefCorr(rvec_f Photon_pt, rvec_f Photon_eta, rvec_i Photon_jetIdx, rvec_i Photon_electronIdx, rvec_f Electron_pt, rvec_f Electron_eta, rvec_i Electron_jetIdx, rvec_i Electron_photonIdx, rvec_f Jet_pt, rvec_f Jet_eta")\
                         .Define("PrefireWeight", "prefCorrs[0]")\
                         .Define("PrefireWeightUp", "prefCorrs[1]")\
                         .Define("PrefireWeightDown", "prefCorrs[2]")

#### btagSF #####


#### metCorrector ####


#### fatJetCorrector ####


#### muonScaleRes ####

#### ht ####

h = df_mht.Histo1D("Muon_effSF")

runtime_error: Template method resolution failed:
  ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void> ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void>::Define(experimental::basic_string_view<char,char_traits<char> > name, experimental::basic_string_view<char,char_traits<char> > expression) =>
    runtime_error: 
RDataFrame: An error occurred during just-in-time compilation. The lines above might indicate the cause of the crash
 All RDF objects that have not run an event loop yet should be considered in an invalid state.

  ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void> ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void>::Define(experimental::basic_string_view<char,char_traits<char> > name, experimental::basic_string_view<char,char_traits<char> > expression) =>
    runtime_error: 
RDataFrame: An error occurred during just-in-time compilation. The lines above might indicate the cause of the crash
 All RDF objects that have not run an event lo

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available
input_line_113:2:86: error: use of undeclared identifier 'IsMC'
auto lambda15 = [](const float var0, const Float_t var1){return puWeight(var0, var1, IsMC)
                                                                                     ^
input_line_117:2:86: error: use of undeclared identifier 'IsMC'
auto lambda15 = [](const float var0, const Float_t var1){return puWeight(var0, var1, IsMC)
                                                                                     ^
input_line_118:2:86: error: use of undeclared identifier 'IsMC'
auto lambd

## Trigger event loop

In [ ]:
%%time
c = ROOT.TCanvas()
h.Draw()
c.Draw()

In [ ]:
nevents = nevents_df.GetValue()
gen_weight = gen_weight_df.GetValue()